OBSERVACIÓ: Si teniu dubtes del codi, contactar amb rdr-contacte@csuc.cat



## OBJECTIU DE L'SCRIPT

L'objectiu principal d'aquest script és extreure els links persistents dels fitxers dels datasets.

In [ ]:
# @title Install or Update Libraries. Click the Run button &#x25B6;
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

def install_packages(b):
    """
    Function to install or update required Python libraries.

    Args:
    b: Button object.

    Returns:
    None
    """
    clear_output(wait=True)
    !pip install --upgrade pip -q
    !pip install pyDataverse -q
    !pip install xlsxwriter -q
    !pip install pandas -q
    print("Libraries have been downloaded or updated.")

# Display installation message
display(HTML("<p style='font-size:14px;'><b>Click the button below to install the libraries.</b></p>"))

# Create installation button
install_button = widgets.Button(description='Install Libraries')
install_button.on_click(install_packages)

# Display the installation button
display(install_button)

In [ ]:
# @title Enter API token, doi (in format doi:10.34810/dataXXX ) and the repository URL. Click the &#x25B6; button to execute cell
from IPython.display import display, FileLink
try:
    from google.colab import files
    import ipywidgets as widgets
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
import os
from pyDataverse.api import NativeApi, DataAccessApi
from pyDataverse.models import Dataverse
import pandas as pd
# Provide input values
token = ""  # @param {type:"string"}
identifier = ""  # @param {type:"string"}
base_url = "https://dataverse.csuc.cat/"  # @param {type:"string"}
doi = identifier
api = NativeApi(base_url,token)
data_api = DataAccessApi(base_url,token)
def filemetadata(base_url, token, doi, filemetadata_keys, filemetadata_values):
    """
    Function to extract metadata for files associated with a dataset identified by its DOI.

    Parameters:
    - base_url: str. Base URL of the Dataverse instance.
    - token: str. API token for authentication.
    - doi: str. DOI of the dataset.
    - filemetadata_keys: list. List to store file metadata keys.
    - filemetadata_values: list. List to store file metadata values.

    Returns:
    - None. Updates the provided lists with extracted file metadata.
    """
    from pyDataverse.api import NativeApi, DataAccessApi
    from pyDataverse.models import Dataverse

    # Instantiate API objects for accessing Dataverse
    api = NativeApi(base_url, token)
    data_api = DataAccessApi(base_url, token)

    try:
        # Retrieve dataset metadata
        dataset = api.get_dataset(doi)

        # Iterate through files and extract metadata
        for i in range(len(dataset.json()['data']['latestVersion']['files'])):
            filemetadata_resp = dataset.json()['data']['latestVersion']['files'][i]['dataFile']
            filemetadata_keys_aux = list(filemetadata_resp.keys())
            filemetadata_values_aux = list(filemetadata_resp.values())
            filemetadata_keys.append(filemetadata_keys_aux)
            filemetadata_values.append(filemetadata_values_aux)
    except KeyError:
        print('There was an error reading metadata for the files of the dataset: ' + doi)
filemetadata_keys_one=[]
filemetadata_values_one=[]
filemetadata(base_url, token, doi,filemetadata_keys_one,filemetadata_values_one)
# Create a DataFrame
df = pd.DataFrame(columns=['id', 'filename'])

# Populate the DataFrame
data = []
for values in filemetadata_values_one:
    data.append({'id': f"https://dataverse.csuc.cat/file.xhtml?fileId={values[0]}", 'filename': values[3]})
df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
# Rename 'id' column to 'Persistent Link'
df.rename(columns={'id': 'Persistent Link'}, inplace=True)
# Save DataFrame to Excel
excel_name = doi.split("/")[-1]
with pd.ExcelWriter(f'{excel_name}.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name=excel_name)

# Download button
files.download(f'{excel_name}.xlsx')